In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

def parse_hmm(fname):
    f = open(fname)
    line=f.readline()
    while line[0]!='#':
        line=f.readline()
    f.readline()
    f.readline()
    f.readline()
    f.readline()
    seq = []
    extras = np.zeros([0,10])
    prob = np.zeros([0,20])
    line = f.readline()
    while line[0:2]!='//':
        lineinfo = line.split()
        seq.append(lineinfo[0])
        probs_ = [2**(-float(lineinfo[i])/1000) if lineinfo[i]!='*' else 0. for i in range(2,22)]
        prob = np.concatenate((prob,np.matrix(probs_)),axis=0)

        line = f.readline()
        lineinfo = line.split()
        extras_ = [2**(-float(lineinfo[i])/1000) if lineinfo[i]!='*' else 0. for i in range(0,10)]
        extras = np.concatenate((extras,np.matrix(extras_)),axis=0)

        line = f.readline()
        assert len(line.strip())==0

        line = f.readline()
    return (''.join(seq),prob,extras)

## HMM Profile Extraction

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/HDA/Benchmark_BinaryML.csv')

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
hmm_matrices = []

for i in df['PDBid']:
    hmm_mat = parse_hmm(f'/content/drive/MyDrive/HDA/Benchmark_HMM/{i}.txt')
    hmm_arr = np.array(hmm_mat[1])
    hmm_matrices.append(hmm_arr)

# Add the list of hmm_arr as a new column 'hmm_matrix' in the DataFrame
df['hmm_matrix'] = hmm_matrices

## Skipxgram Extraction

In [ ]:
import numpy as np

def calculate_sxgbg_features(evolutionary_profile, X):

    L, _ = evolutionary_profile.shape
    sxgbg_matrix = np.zeros((20, 20))

    for i in range(20):
        for j in range(20):
            sxgbg_value = 0.0

            for l in range(1,L - X):
                sxgbg_value += evolutionary_profile[l-1, i] * evolutionary_profile[l + X , j]

            sxgbg_matrix[i, j] = sxgbg_value

    return sxgbg_matrix

### S0G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s0g_matrix = []

for i in df['hmm_matrix']:
    s0g_mat = calculate_sxgbg_features(i, 0)
    s0g_arr = np.array(s0g_mat)
    s0g_matrix.append(s0g_arr)

df['s0g_matrix'] = s0g_matrix

### S1G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s1g_matrix = []

for i in df['hmm_matrix']:
    s1g_mat = calculate_sxgbg_features(i, 1)
    s1g_arr = np.array(s1g_mat)
    s1g_matrix.append(s1g_arr)

df['s1g_matrix'] = s1g_matrix

### S2G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s2g_matrix = []

for i in df['hmm_matrix']:
    s2g_mat = calculate_sxgbg_features(i, 2)
    s2g_arr = np.array(s2g_mat)
    s2g_matrix.append(s2g_arr)

df['s2g_matrix'] = s2g_matrix

### S3G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s3g_matrix = []

for i in df['hmm_matrix']:
    s3g_mat = calculate_sxgbg_features(i, 3)
    s3g_arr = np.array(s3g_mat)
    s3g_matrix.append(s3g_arr)

df['s3g_matrix'] = s3g_matrix

### S4G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s4g_matrix = []

for i in df['hmm_matrix']:
    s4g_mat = calculate_sxgbg_features(i, 4)
    s4g_arr = np.array(s4g_mat)
    s4g_matrix.append(s4g_arr)

df['s4g_matrix'] = s4g_matrix

### S5G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s5g_matrix = []

for i in df['hmm_matrix']:
    s5g_mat = calculate_sxgbg_features(i, 5)
    s5g_arr = np.array(s5g_mat)
    s5g_matrix.append(s5g_arr)

df['s5g_matrix'] = s5g_matrix

### S6G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s6g_matrix = []

for i in df['hmm_matrix']:
    s6g_mat = calculate_sxgbg_features(i, 6)
    s6g_arr = np.array(s6g_mat)
    s6g_matrix.append(s6g_arr)

df['s6g_matrix'] = s6g_matrix

## Evaluation Metrics

In [ ]:
def calculate_accuracy(y_true, y_pred):
    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array((y_pred>0.5).astype(int))

    # Initialize accuracy
    accuracy = 0

    # Calculate accuracy for each instance
    for i in range(len(y_true)):
        # Calculate intersection and union
        intersection = np.sum(np.logical_and(y_true[i], y_pred[i]))
        union = np.sum(np.logical_or(y_true[i], y_pred[i]))

        # Add to total accuracy
        accuracy += intersection / union

    # Calculate average accuracy
    accuracy /= len(y_true)

    return accuracy

In [ ]:
import numpy as np

def norm_accuracy(y_true, y_pred):
    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array((y_pred>0.5).astype(int))

    acc = []
    # Loop over each instance
    for i in range(len(y_true)):
        # Calculate the number of correct predictions for this instance
        print(y_true[i], y_pred[i])
        correct_predictions = np.sum(y_true[i] == y_pred[i])
        print(correct_predictions/5)
        acc.append(correct_predictions/5)

    # Calculate accuracy
    accuracy = sum(acc) / len(y_true)

    return accuracy

## S0G Matrix Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

# Define the model
model_s0g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s0g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s0g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s0g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s0g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 3s 10ms/step - loss: 0.5071 - accuracy: 0.4242 - val_loss: 0.4567 - val_accuracy: 0.4224 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 8ms/step - loss: 0.4739 - accuracy: 0.4221 - val_loss: 0.4741 - val_accuracy: 0.2845 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 7ms/step - loss: 0.4530 - accuracy: 0.4675 - val_loss: 0.4256 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.4121 - accuracy: 0.5238 - val_loss: 0.4102 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4199 - accuracy: 0.5498 - val_loss: 0.4684 - val_accuracy: 0.4138 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.3974 - accuracy: 0.5649 - val_loss: 0.3856 - val_accuracy: 0.5690 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 3s 17ms/step - loss: 0.2652 - accuracy: 0.7749 - val_loss: 0.2427 - val_accuracy: 0.7586 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2082 - accuracy: 0.7879 - val_loss: 0.2091 - val_accuracy: 0.8448 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2130 - accuracy: 0.7944 - val_loss: 0.1995 - val_accuracy: 0.7931 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1854 - accuracy: 0.8139 - val_loss: 0.1987 - val_accuracy: 0.8362 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1776 - accuracy: 0.8182 - val_loss: 0.2223 - val_accuracy: 0.7931 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2008 - accuracy: 0.7965 - val_loss: 0.2395 - val_accuracy: 0.7931 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1904 - accuracy: 0.8143 - val_loss: 0.1785 - val_accuracy: 0.8261 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1510 - accuracy: 0.8553 - val_loss: 0.2097 - val_accuracy: 0.7826 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1564 - accuracy: 0.8359 - val_loss: 0.1886 - val_accuracy: 0.7913 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1358 - accuracy: 0.8639 - val_loss: 0.2147 - val_accuracy: 0.7652 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1445 - accuracy: 0.8315 - val_loss: 0.2747 - val_accuracy: 0.7304 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1366 - accuracy: 0.8553 - val_loss: 0.2136 - val_accuracy: 0.7913 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - lo

## S1G Matrix Model

In [ ]:
# Define the model
model_s1g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s1g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s1g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s1g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s1g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.5453 - accuracy: 0.3961 - val_loss: 0.4711 - val_accuracy: 0.4655 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.4413 - accuracy: 0.5108 - val_loss: 0.3759 - val_accuracy: 0.6810 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 8ms/step - loss: 0.4242 - accuracy: 0.5216 - val_loss: 0.4209 - val_accuracy: 0.5690 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 8ms/step - loss: 0.3692 - accuracy: 0.5779 - val_loss: 0.3673 - val_accuracy: 0.6034 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.3793 - accuracy: 0.5714 - val_loss: 0.3231 - val_accuracy: 0.5948 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 8ms/step - loss: 0.3477 - accuracy: 0.5866 - val_loss: 0.3158 - val_accuracy: 0.7155 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 11ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 10ms/step - loss: 0.3532 - accuracy: 0.6104 - val_loss: 0.4111 - val_accuracy: 0.4914 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 8ms/step - loss: 0.3283 - accuracy: 0.6407 - val_loss: 0.3078 - val_accuracy: 0.6552 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 8ms/step - loss: 0.3394 - accuracy: 0.6017 - val_loss: 0.3247 - val_accuracy: 0.5862 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 10ms/step - loss: 0.3125 - accuracy: 0.6494 - val_loss: 0.3626 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 9ms/step - loss: 0.3089 - accuracy: 0.6082 - val_loss: 0.3629 - val_accuracy: 0.6121 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 8ms/step - loss: 0.3630 - accuracy: 0.6190 - val_loss: 0.3500 - val_accuracy: 0.5948 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 3s 11ms/step - loss: 0.2192 - accuracy: 0.7771 - val_loss: 0.2303 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 8ms/step - loss: 0.2033 - accuracy: 0.7965 - val_loss: 0.1665 - val_accuracy: 0.8276 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 10ms/step - loss: 0.1972 - accuracy: 0.8074 - val_loss: 0.1950 - val_accuracy: 0.7931 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 2s 15ms/step - loss: 0.1802 - accuracy: 0.8117 - val_loss: 0.2611 - val_accuracy: 0.7672 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 3s 22ms/step - loss: 0.2263 - accuracy: 0.7900 - val_loss: 0.2480 - val_accuracy: 0.7155 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 11ms/step - loss: 0.2093 - accuracy: 0.7749 - val_loss: 0.1921 - val_accuracy: 0.8448 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 3s 11ms/step - loss: 0.1391 - accuracy: 0.8661 - val_loss: 0.2984 - val_accuracy: 0.7478 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 10ms/step - loss: 0.1372 - accuracy: 0.8747 - val_loss: 0.1639 - val_accuracy: 0.8174 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 12ms/step - loss: 0.1684 - accuracy: 0.8380 - val_loss: 0.3054 - val_accuracy: 0.6609 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 12ms/step - loss: 0.1159 - accuracy: 0.8726 - val_loss: 0.1341 - val_accuracy: 0.8348 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 2s 15ms/step - loss: 0.1546 - accuracy: 0.8510 - val_loss: 0.2434 - val_accuracy: 0.7478 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 9ms/step - loss: 0.1160 - accuracy: 0.8769 - val_loss: 0.1830 - val_accuracy: 0.8087 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 6ms/step

## S2G Matrix Model

In [ ]:
# Define the model
model_s2g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s2g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s2g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s2g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s2g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.5158 - accuracy: 0.4113 - val_loss: 0.4232 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4645 - accuracy: 0.5022 - val_loss: 0.4288 - val_accuracy: 0.5948 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.4423 - accuracy: 0.5000 - val_loss: 0.4206 - val_accuracy: 0.4483 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 6ms/step - loss: 0.3900 - accuracy: 0.5866 - val_loss: 0.4208 - val_accuracy: 0.4310 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 12ms/step - loss: 0.3821 - accuracy: 0.5433 - val_loss: 0.3770 - val_accuracy: 0.6207 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 9ms/step - loss: 0.3759 - accuracy: 0.5823 - val_loss: 0.3541 - val_accuracy: 0.5862 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 12ms/step - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.1703 - accuracy: 0.8380 - val_loss: 0.1370 - val_accuracy: 0.8957 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1627 - accuracy: 0.8510 - val_loss: 0.2437 - val_accuracy: 0.6783 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1810 - accuracy: 0.8272 - val_loss: 0.2137 - val_accuracy: 0.7739 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1748 - accuracy: 0.8596 - val_loss: 0.2263 - val_accuracy: 0.7565 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1405 - accuracy: 0.8531 - val_loss: 0.1928 - val_accuracy: 0.7826 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1360 - accuracy: 0.8575 - val_loss: 0.3321 - val_accuracy: 0.7217 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - lo

## S3G Matrix Model

In [ ]:
# Define the model
model_s3g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s3g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s3g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s3g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s3g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 10ms/step - loss: 0.5067 - accuracy: 0.4134 - val_loss: 0.4638 - val_accuracy: 0.3707 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 7ms/step - loss: 0.4513 - accuracy: 0.4459 - val_loss: 0.4118 - val_accuracy: 0.4914 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4199 - accuracy: 0.5022 - val_loss: 0.4090 - val_accuracy: 0.4224 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.4125 - accuracy: 0.5108 - val_loss: 0.3775 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.3861 - accuracy: 0.5195 - val_loss: 0.3527 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3746 - accuracy: 0.5346 - val_loss: 0.3474 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.2788 - accuracy: 0.7208 - val_loss: 0.2691 - val_accuracy: 0.6897 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2620 - accuracy: 0.7294 - val_loss: 0.3517 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2702 - accuracy: 0.7013 - val_loss: 0.2654 - val_accuracy: 0.7069 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2517 - accuracy: 0.7424 - val_loss: 0.2833 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2427 - accuracy: 0.7316 - val_loss: 0.3094 - val_accuracy: 0.7155 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2415 - accuracy: 0.7489 - val_loss: 0.2594 - val_accuracy: 0.7586 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 5ms/step - loss: 0.2960 - accuracy: 0.6537 - val_loss: 0.2967 - val_accuracy: 0.6293 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3081 - accuracy: 0.6364 - val_loss: 0.3079 - val_accuracy: 0.6293 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 4ms/step - loss: 0.3279 - accuracy: 0.6255 - val_loss: 0.3657 - val_accuracy: 0.5259 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.3099 - accuracy: 0.5866 - val_loss: 0.2979 - val_accuracy: 0.6034 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2903 - accuracy: 0.6364 - val_loss: 0.2733 - val_accuracy: 0.6552 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2725 - accuracy: 0.7273 - val_loss: 0.2632 - val_accuracy: 0.6897 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.2334 - accuracy: 0.7343 - val_loss: 0.2079 - val_accuracy: 0.7913 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2132 - accuracy: 0.7624 - val_loss: 0.1108 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2161 - accuracy: 0.7451 - val_loss: 0.1565 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1980 - accuracy: 0.7775 - val_loss: 0.1295 - val_accuracy: 0.9217 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1960 - accuracy: 0.7840 - val_loss: 0.1873 - val_accuracy: 0.7826 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2107 - accuracy: 0.7775 - val_loss: 0.1343 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 3s 11ms/step - loss: 0.1633 - accuracy: 0.8251 - val_loss: 0.2058 - val_accuracy: 0.7565 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2020 - accuracy: 0.7927 - val_loss: 0.2347 - val_accuracy: 0.6696 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1990 - accuracy: 0.7775 - val_loss: 0.2225 - val_accuracy: 0.7391 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1489 - accuracy: 0.8488 - val_loss: 0.1805 - val_accuracy: 0.7739 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1587 - accuracy: 0.8380 - val_loss: 0.2461 - val_accuracy: 0.7217 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1434 - accuracy: 0.8380 - val_loss: 0.2273 - val_accuracy: 0.7217 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S4G Matrix Model

In [ ]:
# Define the model
model_s4g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s4g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s4g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s4g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s4g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 9ms/step - loss: 0.5487 - accuracy: 0.3831 - val_loss: 0.4524 - val_accuracy: 0.3448 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 7ms/step - loss: 0.4478 - accuracy: 0.4935 - val_loss: 0.3860 - val_accuracy: 0.6034 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 7ms/step - loss: 0.4253 - accuracy: 0.5108 - val_loss: 0.4012 - val_accuracy: 0.5172 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.4031 - accuracy: 0.5303 - val_loss: 0.3658 - val_accuracy: 0.5690 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.3745 - accuracy: 0.5823 - val_loss: 0.3631 - val_accuracy: 0.5345 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.3513 - accuracy: 0.5736 - val_loss: 0.3441 - val_accuracy: 0.5431 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 5ms/step - loss: 0.3002 - accuracy: 0.6905 - val_loss: 0.2519 - val_accuracy: 0.6983 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2900 - accuracy: 0.6797 - val_loss: 0.3315 - val_accuracy: 0.6121 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2639 - accuracy: 0.7251 - val_loss: 0.2303 - val_accuracy: 0.7672 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2737 - accuracy: 0.7121 - val_loss: 0.2768 - val_accuracy: 0.6724 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2542 - accuracy: 0.7121 - val_loss: 0.2697 - val_accuracy: 0.6810 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2276 - accuracy: 0.7684 - val_loss: 0.2893 - val_accuracy: 0.6466 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 10ms/step - loss: 0.2681 - accuracy: 0.7273 - val_loss: 0.2301 - val_accuracy: 0.7672 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2596 - accuracy: 0.7013 - val_loss: 0.2792 - val_accuracy: 0.6724 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2574 - accuracy: 0.7273 - val_loss: 0.3207 - val_accuracy: 0.6207 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3005 - accuracy: 0.6883 - val_loss: 0.3503 - val_accuracy: 0.5345 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2774 - accuracy: 0.6883 - val_loss: 0.2947 - val_accuracy: 0.6810 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2335 - accuracy: 0.7381 - val_loss: 0.2622 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 7ms/step - loss: 0.2600 - accuracy: 0.7171 - val_loss: 0.1647 - val_accuracy: 0.8435 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 8ms/step - loss: 0.2277 - accuracy: 0.7408 - val_loss: 0.1835 - val_accuracy: 0.8609 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 8ms/step - loss: 0.2325 - accuracy: 0.7451 - val_loss: 0.1504 - val_accuracy: 0.8696 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2378 - accuracy: 0.7408 - val_loss: 0.2164 - val_accuracy: 0.7826 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 8ms/step - loss: 0.2458 - accuracy: 0.7214 - val_loss: 0.1719 - val_accuracy: 0.8087 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.3121 - accuracy: 0.6782 - val_loss: 0.2418 - val_accuracy: 0.6696 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.2161 - accuracy: 0.7754 - val_loss: 0.2462 - val_accuracy: 0.7130 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1851 - accuracy: 0.8099 - val_loss: 0.2638 - val_accuracy: 0.6783 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2133 - accuracy: 0.7603 - val_loss: 0.2437 - val_accuracy: 0.6957 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1873 - accuracy: 0.8013 - val_loss: 0.2246 - val_accuracy: 0.7043 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1856 - accuracy: 0.7970 - val_loss: 0.2546 - val_accuracy: 0.6522 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1711 - accuracy: 0.8359 - val_loss: 0.2499 - val_accuracy: 0.6870 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S5G Matrix Model

In [ ]:
# Define the model
model_s5g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s5g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s5g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s5g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s5g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.5350 - accuracy: 0.4134 - val_loss: 0.4299 - val_accuracy: 0.3793 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4530 - accuracy: 0.4459 - val_loss: 0.4238 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 7ms/step - loss: 0.4365 - accuracy: 0.5130 - val_loss: 0.4182 - val_accuracy: 0.5172 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 8ms/step - loss: 0.3966 - accuracy: 0.5606 - val_loss: 0.3597 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 9ms/step - loss: 0.3931 - accuracy: 0.5606 - val_loss: 0.3568 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 8ms/step - loss: 0.3765 - accuracy: 0.5476 - val_loss: 0.3507 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.1640 - accuracy: 0.8488 - val_loss: 0.1824 - val_accuracy: 0.8087 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1385 - accuracy: 0.8618 - val_loss: 0.1946 - val_accuracy: 0.8261 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1433 - accuracy: 0.8661 - val_loss: 0.1609 - val_accuracy: 0.8348 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1150 - accuracy: 0.8920 - val_loss: 0.1036 - val_accuracy: 0.9043 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1384 - accuracy: 0.8683 - val_loss: 0.3670 - val_accuracy: 0.7217 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1725 - accuracy: 0.8294 - val_loss: 0.1367 - val_accuracy: 0.8870 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - lo

## S6G Matrix Model

In [ ]:
# Define the model
model_s6g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s6g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s6g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s6g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s6g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.5822 - accuracy: 0.4264 - val_loss: 0.4148 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.4555 - accuracy: 0.5216 - val_loss: 0.4029 - val_accuracy: 0.5862 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4327 - accuracy: 0.5303 - val_loss: 0.3993 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4067 - accuracy: 0.5411 - val_loss: 0.3998 - val_accuracy: 0.4741 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.4013 - accuracy: 0.5238 - val_loss: 0.3424 - val_accuracy: 0.6724 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 4ms/step - loss: 0.3883 - accuracy: 0.5693 - val_loss: 0.3506 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 9ms/step - loss: 0.2837 - accuracy: 0.6840 - val_loss: 0.3616 - val_accuracy: 0.5862 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 7ms/step - loss: 0.3000 - accuracy: 0.6688 - val_loss: 0.2927 - val_accuracy: 0.6466 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 8ms/step - loss: 0.2988 - accuracy: 0.6840 - val_loss: 0.2681 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2598 - accuracy: 0.7121 - val_loss: 0.2444 - val_accuracy: 0.7328 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 6ms/step - loss: 0.2508 - accuracy: 0.7338 - val_loss: 0.2314 - val_accuracy: 0.7414 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2464 - accuracy: 0.7078 - val_loss: 0.2317 - val_accuracy: 0.7328 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 5ms/step - loss: 0.2601 - accuracy: 0.7294 - val_loss: 0.5203 - val_accuracy: 0.5345 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2682 - accuracy: 0.7013 - val_loss: 0.2814 - val_accuracy: 0.7069 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2342 - accuracy: 0.7446 - val_loss: 0.2370 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2369 - accuracy: 0.7489 - val_loss: 0.2413 - val_accuracy: 0.7759 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2207 - accuracy: 0.7662 - val_loss: 0.2506 - val_accuracy: 0.7759 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 6ms/step - loss: 0.2377 - accuracy: 0.7532 - val_loss: 0.3204 - val_accuracy: 0.6466 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 3s 10ms/step - loss: 0.2149 - accuracy: 0.7905 - val_loss: 0.2100 - val_accuracy: 0.7652 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 9ms/step - loss: 0.1760 - accuracy: 0.8467 - val_loss: 0.2341 - val_accuracy: 0.7217 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 8ms/step - loss: 0.1732 - accuracy: 0.8229 - val_loss: 0.2995 - val_accuracy: 0.6870 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1694 - accuracy: 0.8337 - val_loss: 0.1875 - val_accuracy: 0.8087 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1741 - accuracy: 0.8488 - val_loss: 0.2842 - val_accuracy: 0.6783 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1613 - accuracy: 0.8315 - val_loss: 0.2283 - val_accuracy: 0.7391 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - l

# Standardizing SkipGram

In [ ]:
# standardize the sxgbg_matrix
import numpy as np

def calculate_sxgbg_stan_features(evolutionary_profile, X):
    L, _ = evolutionary_profile.shape
    sxgbg_matrix = np.zeros((20, 20))

    for i in range(20):
        for j in range(20):
            sxgbg_value = 0.0

            for l in range(1, L - X):
                sxgbg_value += evolutionary_profile[l - 1, i] * evolutionary_profile[l + X, j]

            sxgbg_matrix[i, j] = sxgbg_value

    # Standardize the sxgbg_matrix (z-score normalization)
    mean = np.mean(sxgbg_matrix)
    std = np.std(sxgbg_matrix)

    if std != 0:
        sxgbg_matrix = (sxgbg_matrix - mean) / std
    else:
        sxgbg_matrix = np.zeros_like(sxgbg_matrix)  # Handle the case of zero standard deviation

    return sxgbg_matrix

## S0G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s0g_stan_matrix = []

for i in df['hmm_matrix']:
    s0g_mat = calculate_sxgbg_stan_features(i, 0)
    s0g_arr = np.array(s0g_mat)
    s0g_stan_matrix.append(s0g_arr)

df['s0g_stan_matrix'] = s0g_stan_matrix

## S1G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s1g_stan_matrix = []

for i in df['hmm_matrix']:
    s1g_mat = calculate_sxgbg_stan_features(i, 1)
    s1g_arr = np.array(s1g_mat)
    s1g_stan_matrix.append(s1g_arr)

df['s1g_stan_matrix'] = s1g_stan_matrix

## S2G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s2g_stan_matrix = []

for i in df['hmm_matrix']:
    s2g_mat = calculate_sxgbg_stan_features(i, 2)
    s2g_arr = np.array(s2g_mat)
    s2g_stan_matrix.append(s2g_arr)

df['s2g_stan_matrix'] = s2g_stan_matrix

## S3G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s3g_stan_matrix = []

for i in df['hmm_matrix']:
    s3g_mat = calculate_sxgbg_stan_features(i, 3)
    s3g_arr = np.array(s3g_mat)
    s3g_stan_matrix.append(s3g_arr)

df['s3g_stan_matrix'] = s3g_stan_matrix

## S4G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s4g_stan_matrix = []

for i in df['hmm_matrix']:
    s4g_mat = calculate_sxgbg_stan_features(i, 4)
    s4g_arr = np.array(s4g_mat)
    s4g_stan_matrix.append(s4g_arr)

df['s4g_stan_matrix'] = s4g_stan_matrix

## S5G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s5g_stan_matrix = []

for i in df['hmm_matrix']:
    s5g_mat = calculate_sxgbg_stan_features(i, 5)
    s5g_arr = np.array(s5g_mat)
    s5g_stan_matrix.append(s5g_arr)

df['s5g_stan_matrix'] = s5g_stan_matrix

## S6G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s6g_stan_matrix = []

for i in df['hmm_matrix']:
    s6g_mat = calculate_sxgbg_stan_features(i, 6)
    s6g_arr = np.array(s6g_mat)
    s6g_stan_matrix.append(s6g_arr)

df['s6g_stan_matrix'] = s6g_stan_matrix

## S0G Stan Matrix Model

In [ ]:
# Define the model
model_s0g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s0g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s0g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s0g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s0g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 3s 13ms/step - loss: 0.4216 - accuracy: 0.4827 - val_loss: 0.3858 - val_accuracy: 0.5345 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 10ms/step - loss: 0.3349 - accuracy: 0.6082 - val_loss: 0.4075 - val_accuracy: 0.5862 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 2s 16ms/step - loss: 0.3076 - accuracy: 0.6667 - val_loss: 0.3237 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 2s 14ms/step - loss: 0.2592 - accuracy: 0.7165 - val_loss: 0.3150 - val_accuracy: 0.6466 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2459 - accuracy: 0.7597 - val_loss: 0.2958 - val_accuracy: 0.6897 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 8ms/step - loss: 0.2109 - accuracy: 0.8052 - val_loss: 0.2871 - val_accuracy: 0.6983 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 9ms/step 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 4s 14ms/step - loss: 0.1735 - accuracy: 0.8420 - val_loss: 0.1107 - val_accuracy: 0.9052 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 2s 18ms/step - loss: 0.1556 - accuracy: 0.8528 - val_loss: 0.0904 - val_accuracy: 0.9310 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 2s 18ms/step - loss: 0.1129 - accuracy: 0.8961 - val_loss: 0.1152 - val_accuracy: 0.8966 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 12ms/step - loss: 0.1003 - accuracy: 0.9026 - val_loss: 0.1441 - val_accuracy: 0.8621 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 9ms/step - loss: 0.1079 - accuracy: 0.8961 - val_loss: 0.0892 - val_accuracy: 0.9310 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 8ms/step - loss: 0.0947 - accuracy: 0.9069 - val_loss: 0.0962 - val_accuracy: 0.9138 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 8ms/step 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 5s 12ms/step - loss: 0.1138 - accuracy: 0.9006 - val_loss: 0.0880 - val_accuracy: 0.9217 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 9ms/step - loss: 0.0878 - accuracy: 0.9050 - val_loss: 0.1082 - val_accuracy: 0.8870 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 6ms/step - loss: 0.0857 - accuracy: 0.9266 - val_loss: 0.0889 - val_accuracy: 0.9043 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 8ms/step - loss: 0.0709 - accuracy: 0.9395 - val_loss: 0.0857 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 8ms/step - loss: 0.0652 - accuracy: 0.9374 - val_loss: 0.1005 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 8ms/step - loss: 0.0510 - accuracy: 0.9417 - val_loss: 0.0983 - val_accuracy: 0.8870 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step - l

## S1G Stan Matrix Model

In [ ]:
# Define the model
model_s1g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s1g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s1g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s1g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s1g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 8ms/step - loss: 0.4061 - accuracy: 0.4892 - val_loss: 0.3667 - val_accuracy: 0.4828 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 8ms/step - loss: 0.3441 - accuracy: 0.5952 - val_loss: 0.3301 - val_accuracy: 0.6983 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 12ms/step - loss: 0.3014 - accuracy: 0.6775 - val_loss: 0.3316 - val_accuracy: 0.6293 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 12ms/step - loss: 0.2639 - accuracy: 0.6926 - val_loss: 0.2860 - val_accuracy: 0.7155 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 11ms/step - loss: 0.2346 - accuracy: 0.7554 - val_loss: 0.2788 - val_accuracy: 0.7414 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 11ms/step - loss: 0.2304 - accuracy: 0.7554 - val_loss: 0.2892 - val_accuracy: 0.6810 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 1s 6ms/step - loss: 0.2420 - accuracy: 0.7359 - val_loss: 0.1960 - val_accuracy: 0.7759 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2007 - accuracy: 0.7944 - val_loss: 0.2353 - val_accuracy: 0.7069 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1872 - accuracy: 0.7987 - val_loss: 0.2256 - val_accuracy: 0.7672 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1525 - accuracy: 0.8485 - val_loss: 0.2285 - val_accuracy: 0.7672 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1411 - accuracy: 0.8506 - val_loss: 0.2405 - val_accuracy: 0.7328 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1522 - accuracy: 0.8333 - val_loss: 0.2295 - val_accuracy: 0.7586 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 3s 11ms/step - loss: 0.1100 - accuracy: 0.8942 - val_loss: 0.1073 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0954 - accuracy: 0.9071 - val_loss: 0.0877 - val_accuracy: 0.8870 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0812 - accuracy: 0.9266 - val_loss: 0.0990 - val_accuracy: 0.8522 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0783 - accuracy: 0.9244 - val_loss: 0.1102 - val_accuracy: 0.8696 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0698 - accuracy: 0.9352 - val_loss: 0.1687 - val_accuracy: 0.8435 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0665 - accuracy: 0.9503 - val_loss: 0.0917 - val_accuracy: 0.8870 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - l

## S2G Stan Matrix Model

In [ ]:
# Define the model
model_s2g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s2g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s2g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s2g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s2g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.4158 - accuracy: 0.4848 - val_loss: 0.3475 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.3314 - accuracy: 0.6061 - val_loss: 0.3391 - val_accuracy: 0.5345 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2869 - accuracy: 0.7056 - val_loss: 0.3460 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2814 - accuracy: 0.7100 - val_loss: 0.3943 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2457 - accuracy: 0.7294 - val_loss: 0.4304 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2204 - accuracy: 0.7684 - val_loss: 0.3429 - val_accuracy: 0.6466 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 8ms/step - loss: 0.2043 - accuracy: 0.7900 - val_loss: 0.1381 - val_accuracy: 0.8534 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1620 - accuracy: 0.8333 - val_loss: 0.1485 - val_accuracy: 0.8621 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1346 - accuracy: 0.8550 - val_loss: 0.1223 - val_accuracy: 0.8879 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1221 - accuracy: 0.8723 - val_loss: 0.1915 - val_accuracy: 0.8017 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1218 - accuracy: 0.8723 - val_loss: 0.1606 - val_accuracy: 0.8103 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0954 - accuracy: 0.9026 - val_loss: 0.1246 - val_accuracy: 0.8621 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0927 - accuracy: 0.9006 - val_loss: 0.0957 - val_accuracy: 0.8696 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0687 - accuracy: 0.9330 - val_loss: 0.0602 - val_accuracy: 0.9565 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 4ms/step - loss: 0.0660 - accuracy: 0.9266 - val_loss: 0.1037 - val_accuracy: 0.8957 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0678 - accuracy: 0.9330 - val_loss: 0.1161 - val_accuracy: 0.8870 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0506 - accuracy: 0.9417 - val_loss: 0.0886 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 4ms/step - loss: 0.0497 - accuracy: 0.9525 - val_loss: 0.0653 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

## S3G Stan Matrix Model

In [ ]:
# Define the model
model_s3g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s3g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s3g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s3g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s3g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 8ms/step - loss: 0.4436 - accuracy: 0.4610 - val_loss: 0.3591 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 2s 15ms/step - loss: 0.3412 - accuracy: 0.6320 - val_loss: 0.3475 - val_accuracy: 0.5345 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 8ms/step - loss: 0.3081 - accuracy: 0.6190 - val_loss: 0.2901 - val_accuracy: 0.6466 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2612 - accuracy: 0.7121 - val_loss: 0.4022 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2460 - accuracy: 0.7359 - val_loss: 0.3135 - val_accuracy: 0.6034 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2267 - accuracy: 0.7706 - val_loss: 0.3746 - val_accuracy: 0.5948 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.1437 - accuracy: 0.8596 - val_loss: 0.1572 - val_accuracy: 0.8261 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1260 - accuracy: 0.8812 - val_loss: 0.1707 - val_accuracy: 0.8174 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1266 - accuracy: 0.8855 - val_loss: 0.1931 - val_accuracy: 0.8174 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1083 - accuracy: 0.8985 - val_loss: 0.1600 - val_accuracy: 0.8348 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0854 - accuracy: 0.9179 - val_loss: 0.1825 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0737 - accuracy: 0.9201 - val_loss: 0.1808 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

## S4G Stan Matrix Model

In [ ]:
# Define the model
model_s4g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s4g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s4g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s4g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s4g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.4307 - accuracy: 0.4654 - val_loss: 0.3379 - val_accuracy: 0.6207 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.3455 - accuracy: 0.6126 - val_loss: 0.3181 - val_accuracy: 0.6810 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.3057 - accuracy: 0.6580 - val_loss: 0.3024 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2644 - accuracy: 0.7121 - val_loss: 0.3381 - val_accuracy: 0.5862 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2237 - accuracy: 0.7597 - val_loss: 0.3310 - val_accuracy: 0.5862 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1940 - accuracy: 0.8030 - val_loss: 0.3401 - val_accuracy: 0.6466 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 3s 11ms/step - loss: 0.0835 - accuracy: 0.9179 - val_loss: 0.0496 - val_accuracy: 0.9391 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 8ms/step - loss: 0.0833 - accuracy: 0.9201 - val_loss: 0.0666 - val_accuracy: 0.9391 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 8ms/step - loss: 0.0781 - accuracy: 0.9330 - val_loss: 0.0843 - val_accuracy: 0.8957 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0690 - accuracy: 0.9244 - val_loss: 0.1058 - val_accuracy: 0.8870 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0717 - accuracy: 0.9244 - val_loss: 0.0922 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0476 - accuracy: 0.9460 - val_loss: 0.1384 - val_accuracy: 0.8435 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - l

## S5G Stan Matrix Model

In [ ]:
# Define the model
model_s5g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s5g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s5g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s5g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s5g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.4611 - accuracy: 0.4286 - val_loss: 0.3748 - val_accuracy: 0.5172 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.3272 - accuracy: 0.6407 - val_loss: 0.3354 - val_accuracy: 0.6034 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 6ms/step - loss: 0.2935 - accuracy: 0.6797 - val_loss: 0.4702 - val_accuracy: 0.4914 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 8ms/step - loss: 0.2622 - accuracy: 0.7100 - val_loss: 0.3179 - val_accuracy: 0.5948 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 8ms/step - loss: 0.2272 - accuracy: 0.7554 - val_loss: 0.2893 - val_accuracy: 0.6897 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 8ms/step - loss: 0.2280 - accuracy: 0.7511 - val_loss: 0.2691 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 9ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 5ms/step - loss: 0.0873 - accuracy: 0.9158 - val_loss: 0.1471 - val_accuracy: 0.8522 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0647 - accuracy: 0.9330 - val_loss: 0.0461 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 4ms/step - loss: 0.0560 - accuracy: 0.9374 - val_loss: 0.0637 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0573 - accuracy: 0.9395 - val_loss: 0.0672 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0933 - accuracy: 0.9028 - val_loss: 0.1600 - val_accuracy: 0.8609 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 4ms/step - loss: 0.0597 - accuracy: 0.9395 - val_loss: 0.0861 - val_accuracy: 0.9217 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

## S5G Stan Matrix LSTM Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

# Define the model
model_lstmsn = keras.Sequential([
    keras.layers.Input(shape=(400,1)),
    keras.layers.LSTM(256, return_sequences=True, kernel_initializer = 'he_normal'),
    keras.layers.LSTM(128, return_sequences=True, kernel_initializer = 'he_normal'),  # Another LSTM layer with 64 units
    keras.layers.Flatten(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu', kernel_initializer = 'he_normal'),     # Fully connected layer
    keras.layers.Dense(5, activation='sigmoid')    # Output layer with sigmoid activation for multi-label classification
])

train_features = np.array(df['s5g_stan_matrix'].tolist())
train_features = train_features.reshape(-1, 400,1)

train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_lstmsn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=7, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=4, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_lstmsn.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_lstmsn.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy+avg_precision+avg_recall+f1+avg_acc_norm++avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 16s 49ms/step - loss: 0.5222 - accuracy: 0.4437 - val_loss: 0.3927 - val_accuracy: 0.5086 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 4s 37ms/step - loss: 0.3944 - accuracy: 0.5195 - val_loss: 0.3519 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 4s 39ms/step - loss: 0.3671 - accuracy: 0.5823 - val_loss: 0.3428 - val_accuracy: 0.6293 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 5s 40ms/step - loss: 0.3353 - accuracy: 0.6082 - val_loss: 0.3354 - val_accuracy: 0.6293 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 4s 37ms/step - loss: 0.3201 - accuracy: 0.6494 - val_loss: 0.3282 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 4s 37ms/step - loss: 0.2908 - accuracy: 0.6861 - val_loss: 0.3097 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 5s 42ms/s

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


116/116 [==============================] - 9s 46ms/step - loss: 0.2128 - accuracy: 0.8247 - val_loss: 0.0974 - val_accuracy: 0.9310 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 5s 43ms/step - loss: 0.1262 - accuracy: 0.8766 - val_loss: 0.1077 - val_accuracy: 0.9397 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 4s 38ms/step - loss: 0.1322 - accuracy: 0.8788 - val_loss: 0.1602 - val_accuracy: 0.8621 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 4s 38ms/step - loss: 0.0895 - accuracy: 0.9177 - val_loss: 0.1239 - val_accuracy: 0.8966 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 5s 42ms/step - loss: 0.0617 - accuracy: 0.9394 - val_loss: 0.1584 - val_accuracy: 0.8793 - lr: 0.0010
Epoch 6/30
115/116 [============================>.] - ETA: 0s - loss: 0.0689 - accuracy: 0.9326
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
116/116 [==============================] - 4s 37ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


116/116 [==============================] - 10s 51ms/step - loss: 0.0819 - accuracy: 0.9158 - val_loss: 0.0400 - val_accuracy: 0.9652 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 4s 37ms/step - loss: 0.0525 - accuracy: 0.9503 - val_loss: 0.0459 - val_accuracy: 0.9565 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 4s 38ms/step - loss: 0.0399 - accuracy: 0.9568 - val_loss: 0.0604 - val_accuracy: 0.9391 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 5s 41ms/step - loss: 0.0343 - accuracy: 0.9546 - val_loss: 0.1036 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 5/30
115/116 [============================>.] - ETA: 0s - loss: 0.0658 - accuracy: 0.9391
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
116/116 [==============================] - 4s 38ms/step - loss: 0.0654 - accuracy: 0.9395 - val_loss: 0.1003 - val_accuracy: 0.8957 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 4s 37ms/step - 

## S5G Stan XGBoost Matrix Model

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Define the XGBoost classifier
model_xgb = xgb.XGBClassifier()

# Load your data and pr0.0.eprocess it
train_features = np.array(df['s5g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Fit the XGBoost model to the training data
    model_xgb.fit(X_train, y_train)

    # Make predictions on the current test sample
    y_pred = model_xgb.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy+avg_precision+avg_recall+f1+avg_acc_norm++avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [1 0 0 0 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 1 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 1 0 0 0] [0 0 0 0 0]
0.8
[0 0 1 0 0] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [1 0 0 0 0]
0.6
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [1 0 0 1 0]
0.8
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 1 0 0 0] [0 1 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 1 0] [1 0 0 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [0 0 0 0 1]
0.6
[1 0 0 0 0] [0 0 0 0 1]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 1]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 1 0 0 0] [0 1 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 1]
0.8
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 1] [1 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 1 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [1 0 0 0 1]
0.8
[1 0 0 0 0] [0 0 0 0 1]
0.6
[1 0 0 0 1] [0 0 0 0 0]
0.6
[1 0 0 0 0] [1 0 0 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S5G Stan KNN Matrix Model

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import KFold

# Define the k-NN classifier
model_knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) as needed

# Load your data and preprocess it
train_features = np.array(df['s5g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Fit the k-NN model to the training data
    model_knn.fit(X_train, y_train)

    # Make predictions on the current test sample
    y_pred = model_knn.predict(X_test)

    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy+avg_precision+avg_recall+f1+avg_acc_norm++avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 0 1] [1 0 0 0 0]
0.6
[1 0 0 0 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 1 0 0 0] [0 1 0 0 0]
1.0
[0 0 1 0 0] [1 0 0 0 0]
0.6
[0 0 0 1 0] [1 0 0 0 0]
0.6
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 0 1] [1 0 0 0 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [0 0 0 0 1]
0.6
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [1 0 0 0 0]
0.6
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 1 0]
0.6
[0 1 0 0 0] [0 1 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 0 1]
0.6
[0 0 0 1 0] [0 0 0 1

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0 0 0 1 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 1 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 1 0] [0 0 0 1 0]
0.8
[1 0 0 0 0] [0 1 0 0 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 1 0 0 0] [0 0 0 1 0]
0.6
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 1 0 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 1 0 0] [1 0 0 0 0]
0.6
[0 1 0 0 0] [0 1 0 0 0]
1.0
[0 1 0 0 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 1 0]
0.6
[0 0 0 1 0] [0 0 0 0 1]
0.6
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 1 0 0 0]
0.6
[0 0 0 0 1] [1 0 0 0 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [1 0 0 0 0]
0.6
[0 0 0 1 0] [1 0 0 0 0]
0.6
[0 0 0 1 0] [1 0 0 0 0]
0.6
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 1 0]
0.6
[0 0 0 1 0] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0

## S6G Stan Matrix Model

In [ ]:
# Define the model
model_s6g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s6g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s6g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s6g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s6g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 3s 11ms/step - loss: 0.4130 - accuracy: 0.4762 - val_loss: 0.3344 - val_accuracy: 0.6552 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 8ms/step - loss: 0.3450 - accuracy: 0.5909 - val_loss: 0.3067 - val_accuracy: 0.6552 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 8ms/step - loss: 0.2927 - accuracy: 0.6645 - val_loss: 0.3298 - val_accuracy: 0.6121 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 9ms/step - loss: 0.2698 - accuracy: 0.7078 - val_loss: 0.2835 - val_accuracy: 0.6983 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2317 - accuracy: 0.7749 - val_loss: 0.3321 - val_accuracy: 0.5948 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2155 - accuracy: 0.7727 - val_loss: 0.3373 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.0885 - accuracy: 0.9028 - val_loss: 0.0574 - val_accuracy: 0.9391 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0701 - accuracy: 0.9266 - val_loss: 0.1311 - val_accuracy: 0.8522 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0956 - accuracy: 0.8920 - val_loss: 0.1520 - val_accuracy: 0.8261 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.0765 - accuracy: 0.9136 - val_loss: 0.0999 - val_accuracy: 0.8696 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 4ms/step - loss: 0.0681 - accuracy: 0.9244 - val_loss: 0.0900 - val_accuracy: 0.9043 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 4ms/step - loss: 0.0540 - accuracy: 0.9568 - val_loss: 0.0846 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo